# Converter datas pré gregorianas.


Necessita de 

    pip install convertdate

Exº Dehergne, entrada 628 Francisco Pérez 


 [http://timelink.uc.pt/mhk/china/id/rp-87](http://timelink.uc.pt/mhk/china/id/rp-87)

 M. 12 févr. 1583 (ancien style) = 22 février (nouveau style de la réforme introduite à Rome en oct. 1582),

In [2]:
from convertdate import julian
from convertdate import gregorian

julian.to_gregorian(1583,2,12)

(1583, 2, 22)

In [11]:
print(julian.to_gregorian(1582,9,24))
print(julian.to_gregorian(1582,10,5))

(1582, 10, 4)
(1582, 10, 15)


In [14]:
from convertdate import gregorian

gregorian.monthcalendar(1582,10)

[[None, None, None, None, None, 1, 2],
 [3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16],
 [17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30],
 [31, None, None, None, None, None]]

# Testing mhk_users database

In [1]:
from timelink.mhk.utilities import get_connection_string, get_engine
from sqlalchemy import MetaData, Table
from sqlalchemy.orm import declarative_base, sessionmaker

db = 'mhk_users'
mhk_users = MetaData()
Base = declarative_base()
connection_string = get_connection_string(db)
engine = get_engine(db)
engine.echo = False     # set to False to prevent SQL logging
Session = sessionmaker(bind=engine)
comments_table = Table('comments', mhk_users, autoload_with=engine)
users_table = Table('users', mhk_users, autoload_with=engine)
print('Connected to',db)


Connected to mhk_users


Process the comments

In [2]:
from timelink.mhk.utilities import get_dbnames

print("MHK databases:",get_dbnames())

MHK databases: ['ilhavo', 'ucprosopd', 'ucprosop', 'anaisabel', 'soure_cloud', 'ucalumni', 'china', 'mhk']


In [3]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = ('mhk','ucalumni')
db_main = get_db(db_spec)

In [4]:
from sqlalchemy import select
from ucalumni.aluno import get_aluno_from_db

name=""
change=""
 
template_for_issue = f""" 

Title: Gralha no Registo: https://pesquisa.auc.uc.pt/details?id={id}
**Identificação do registo**

1. Localize o registo no "Índice de alunos da Universidade de Coimbra"
2. Anote o url do registo na linha abaixo

https://pesquisa.auc.uc.pt/details?id={id}
{name}

**Correção**

{change}

-----

 **Não se esqueça de colocar o URL do registo no título**

"""

print(comments_table.c)
stmt = select(comments_table.c.entity_id,comments_table.c.title,comments_table.c.text).where(comments_table.c.title.like('CORRIGIR%'))
with Session() as session:
    result = session.execute(stmt)
    for r in result:
        aluno = get_aluno_from_db(r.entity_id,db_main)
        name = aluno.nome
        id = r.entity_id
        title = r.title
        change = r.text.split('\n')
        print(r.title)
        print(template_for_issue)
        print()
       

NameError: name 'name' is not defined

## Source files

In [ ]:
from sqlalchemy import select
from pathlib import Path

from timelink.mhk.models.base import Source


kleio_files = [f.stem for f in list(Path('../sources').rglob('*.cli'))]
print("Number of kleio_files:", len(kleio_files))

stmt = select(Source.id,Source.updated)

with Session() as session:
    imported_sources = session.execute(stmt)
    sources_in_db = [s.id for s in imported_sources]
    print("Number of imported files:",len(sources_in_db))
    print("Files not in the database:", len(set(kleio_files)-set(sources_in_db)))
    for source in sorted(list(set(kleio_files)-set(sources_in_db))):
        print(source,end=' ')
    print()
    print("Imported sources with no file found:", len(set(sources_in_db)-set(kleio_files)))
    for source in sorted(list(set(sources_in_db)-set(kleio_files))):
        print(source,end=' ')
    print()